# Transaction Data Collection from the Ethereum Blockchain

## EIP-1559
- Date: August 5, 2021
- Block number: 12,965,000
- [Ethereum JSON-RPC Specification](https://ethereum.github.io/execution-apis/api-documentation/)
- [JSON RPC API](https://ethereum.org/en/developers/docs/apis/json-rpc/)
- [EIP-1559 Analysis Arxiv](https://github.com/SciEcon/EIP1559)

## Layer 2 Solutions Launch Dates
Source: [L2BEAT](https://l2beat.com/scaling/tvl)
1. Optimism is live on: January 16, 2021
2. Arbitrum is live on: August 31, 2021

In [1]:
import datetime
import csv
import os
import re
import glob
from datetime import datetime
import time

import pandas as pd
import dask.dataframe as dd
from collections import defaultdict

In [2]:
def timestamp(date_string):
    """
    Convert a date string to a Unix timestamp.
    
    Args:
        date_string (str): The date string in 'YYYY-MM-DD' format.
    
    Returns:
        int: The Unix timestamp corresponding to the date string.
    """
    dt = datetime.strptime(date_string, "%Y-%m-%d")
    return int(dt.timestamp())


### Merge the eth tx data

In [ ]:
# Directory where your files are stored
data_dir = "../data/"

# Filename pattern
filename_pattern = "eth_transaction_data_{}.csv"

# Find all filenames in the directory
all_files = os.listdir(data_dir)

# Extract dates from filenames and convert to datetime
dates = [datetime.strptime(re.search(r'\d{4}-\d{2}-\d{2}', file).group(), "%Y-%m-%d") for file in all_files if re.search(r'\d{4}-\d{2}-\d{2}', file)]

# Find start and end dates
start_date = min(dates)
end_date = max(dates)

print(start_date, end_date)

In this script above:

1. The `os.listdir` function is used to retrieve all the filenames in the directory.
2. The `re.search` function is used to extract the date strings from the filenames using a regular expression that matches the date format (yyyy-mm-dd).
3. The `datetime.strptime` function is used to convert the date strings to datetime objects.
4. The `min` and `max` functions are used to find the start and end dates.

In [ ]:
# Flag to indicate whether it's the first file
first_file = True

# Create or open the final CSV file in append mode
with open('../data/merged_eth_transaction_data.csv', 'a') as singleFile:
    for single_date in pd.date_range(start_date, end_date):
        filename = os.path.join(data_dir, filename_pattern.format(single_date.strftime("%Y-%m-%d")))
        
        if os.path.isfile(filename):  # if the file exists
            df = pd.read_csv(filename, dtype={5: float})
            # Write data to file
            if first_file:  # If it's the first file
                df.to_csv(singleFile, header=True)  # Write with header
                first_file = False  # After the first file, set this flag to False
            else:
                df.to_csv(singleFile, header=False, mode='a')  # If not the first file, write without header

In this script for merging transtion data:
1. It iterates over the date range, reads the data for each date into a DataFrame, and appends it to the final CSV file.
2. a flag `first_file` is used to check if the current file is the first one. If it is, the script writes the DataFrame to the CSV file with headers. For subsequent files, the DataFrame is written without headers. The `mode='a'` argument to `to_csv` is used to append the data to the existing file.

### Data preprocessing
Given the large size of your data, traditional Python tools like Pandas might not be able to handle it efficiently due to memory constraints. Fortunately, there are tools and libraries built specifically for handling larger-than-memory datasets, such as Dask and Vaex.

1. **Dask**: Dask is a flexible library for parallel computing in Python that's built on top of existing Python APIs and data structures, like NumPy arrays and Pandas DataFrames. It can handle larger-than-memory computations by breaking them down into smaller tasks, executing these tasks in parallel and combining the results.


2. **Vaex**: Vaex is another library for handling large datasets. Like Dask, it performs lazy evaluations and only reads in data when necessary. Its API is also similar to Pandas, which makes it easy to use if you're familiar with Pandas:

In either case, you should ensure that your machine has enough storage to hold the intermediate results of the computations. Also, both Dask and Vaex make use of multiple cores, so having a multi-core machine can speed up the computations.

In Dask, the way to read and modify columns is similar to pandas. You can use Dask's `read_csv` function to read the CSV file and then apply the string operations to the column names using the `rename` method. Here's how you can adapt your code to use Dask:

In [4]:
# Specify the data types
dtypes = {
    'Transaction Identifier': str,
    'Transaction Status': str,
    'Sender\'s Address': str,
    'Transaction Type': str,
    'Transaction EIP-1559 Type': str
}

# Read the CSV file into a DataFrame
df = dd.read_csv('../data/merged_eth_transaction_data.csv', dtype=dtypes)
print(df.columns)
print(df.dtypes)

# If 'unnamed: 0' column exists, drop it
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Format column names to replace spaces with '_' and replace single quotes with "".
df = df.rename(columns={col: col.lower().replace(' ', '_').replace("'", "") for col in df.columns})

print(df.columns)
print(df.dtypes)

Index(['Unnamed: 0', 'Transaction Identifier', 'Block Number',
       'Transaction Timestamp', 'Transaction Status', 'Gas Price',
       'Transaction Fee', 'Sender's Address', 'Transaction Type',
       'Transaction EIP-1559 Type'],
      dtype='object')
Unnamed: 0                     int64
Transaction Identifier        object
Block Number                   int64
Transaction Timestamp          int64
Transaction Status            object
Gas Price                      int64
Transaction Fee              float64
Sender's Address              object
Transaction Type              object
Transaction EIP-1559 Type     object
dtype: object
Index(['transaction_identifier', 'block_number', 'transaction_timestamp',
       'transaction_status', 'gas_price', 'transaction_fee', 'senders_address',
       'transaction_type', 'transaction_eip-1559_type'],
      dtype='object')
transaction_identifier        object
block_number                   int64
transaction_timestamp          int64
transaction_statu

Dask's `read_csv` function returns a Dask DataFrame, which is a large parallel DataFrame composed of smaller pandas DataFrames, split along the index. These pandas DataFrames may live on disk for larger-than-memory computing on a single machine, or on many different machines in a cluster.

One important thing to remember is that Dask uses lazy execution, which means it only executes tasks when absolutely necessary. So if you're not seeing any output or the code seems to be running very quickly, it's likely because Dask hasn't actually performed the computations yet.

To actually perform the computations and get the result, you need to call the `compute` method. However, be careful when using `compute` with large datasets, because it loads the result into memory, which could cause your program to run out of memory if the result is too large. In your case, you won't need to use `compute` until you actually need to see the modified DataFrame.

In [5]:
df.head(10)

,transaction_identifier,block_number,transaction_timestamp,transaction_status,gas_price,transaction_fee,senders_address,transaction_type,transaction_eip-1559_type
0,0x804bfa175a2a32b3ed330c7c642ac2210784e1646e39...,11794239,1612501201,Success,269000000000,2.824500e+16,0xb0da6794da4E6f7244B96256AdB8973D07428a20,ERC20 Transfer,Legacy
1,0x2e9c29c7b00fedbf30d0e3f4c4d4039da468b46bdb27...,11794239,1612501201,Success,246015000000,1.833599e+16,0x274F3c32C90517975e29Dfc209a23f315c1e5Fc7,ERC20 Transfer,Legacy
2,0xb78c319c2e68e82be6a0fb34cd49c17e65f1304ea7dc...,11794239,1612501201,Success,209000000000,4.389000e+15,0xf6292B422EaA44165519480Ebc232eEfB9F0a47f,Simple Ether Transfer,Legacy
3,0x9e26892415c6643b1728a5c6048a2521346092287276...,11794239,1612501201,Success,209000000000,1.463000e+17,0xB630D90e3EBE206d2C2A36C060D14812E320862e,Interaction with a Contract,Legacy
4,0xa6798928631b5e101b5dd27d123122b125e8ce5f9a6b...,11794239,1612501201,Success,207000000000,4.347000e+15,0x741c8EE1d80738dBff66cb914cF74763cc7a0f89,Simple Ether Transfer,Legacy
5,0x16286165e0637f5acbe2cbb2c565e3d54aa463431616...,11794239,1612501201,Success,209000000000,4.389000e+15,0x741c8EE1d80738dBff66cb914cF74763cc7a0f89,Simple Ether Transfer,Legacy
6,0x9cd1cf12a60507743a89f2c3d367333bc5a89bc51adb...,11794239,1612501201,Success,207000000000,5.175000e+16,0x1A579A8FeCE2dcA0E74A12Cc350220178A353D25,ERC20 Transfer,Legacy
7,0x665ff9b7c7b89fe8793eda31aecb7ff01b9723e4b7d3...,11794239,1612501201,Success,207000000000,5.175000e+16,0x662F7fEaC9401d403Ddb0972ff85270071009b9F,ERC20 Transfer,Legacy
8,0x20f2c1f815341f6fa822242dc7b2aad5dbe92d1ef474...,11794239,1612501201,Success,207000000000,5.175000e+16,0x55ac3377c5B65d03866895f81B241eaF18221A8D,ERC20 Transfer,Legacy
9,0xa9eec9ad774dce95e9395f9787b080437bc081396af2...,11794239,1612501201,Success,207000000000,4.347000e+15,0x6fAf75112F9Ee5e7A40328F8e64041c4399a2Dc8,Simple Ether Transfer,Legacy


### An alternative approach to merge data from multiple csv files
The script below is more suitable for a smaller number of files or if your machine has a large amount of memory available. We will use this script to merge eth net supply data for each block.

In [18]:
# Set the directory you want to start from
rootDir = '../data/'  # current directory; adjust it if needed

# Find all eth net supply CSV files in the directory
all_files = glob.glob(os.path.join(rootDir, "eth_net_supply_*.csv"))

# Use sort to ensure files are concatenated in the correct order
all_files.sort()

# Use a generator expression to lazily read and concatenate files
merged_data = pd.concat((pd.read_csv(file) for file in all_files))

dtypes = {
    'block_number': 'int',
    'static_reward': 'float',
    'uncle_reward': 'float',
    'inclusion_reward': 'float',
    'base_fee': 'int',
    'gas_used': 'int',
    'burned_fee': 'int',
    'net_supply_increase': 'float'
}

# Convert types for each column
for column, dtype in dtypes.items():
    # Convert the column to numeric values, making non-convertible values NaN
    merged_data[column] = pd.to_numeric(merged_data[column], errors='coerce')

    # Now, drop the rows where we got NaN values due to failed conversion
    merged_data = merged_data[merged_data[column].notna()]

    # And finally, convert the column to the target type
    merged_data[column] = merged_data[column].astype(dtype)

merged_data.to_csv('merged_eth_net_supply.csv', index=False)

/tmp/ipykernel_8766/4060746378.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.concat((pd.read_csv(file) for file in all_files))
/tmp/ipykernel_8766/4060746378.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.concat((pd.read_csv(file) for file in all_files))
/tmp/ipykernel_8766/4060746378.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.concat((pd.read_csv(file) for file in all_files))
/tmp/ipykernel_8766/4060746378.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.concat((pd.read_csv(file) for file in all_files))
/tmp/ipykernel_8766/4060746378.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option

In [19]:
print(merged_data.columns)
print(merged_data.dtypes)

Index(['block_number', 'static_reward', 'uncle_reward', 'inclusion_reward',
       'base_fee', 'gas_used', 'burned_fee', 'net_supply_increase'],
      dtype='object')
block_number             int64
static_reward          float64
uncle_reward           float64
inclusion_reward       float64
base_fee                 int64
gas_used                 int64
burned_fee               int64
net_supply_increase    float64
dtype: object


In [20]:
def extract_unique_senders(input_file, output_file, sender_column):
    """
    Function to extract unique sender addresses from a large CSV file of transaction data using Dask.

    Parameters:
    input_file (str): Path to the input CSV file representing transaction data.
    output_file (str): Path to the output CSV file where unique sender addresses will be saved.
    sender_column (str): The name of the column in the input file that contains the sender's addresses.

    Returns:
    None
    """

    # Read in data using Dask's read_csv function
    # Dask is a parallel computing library that allows us to work with large datasets
    # The read_csv function works similarly to pandas' read_csv, but it performs the operations lazily
    df = dd.read_csv(input_file)

    # Get unique sender's addresses
    # drop_duplicates returns the unique values in the sender_column
    unique_senders = df[sender_column].drop_duplicates()

    # Compute the result and save to a new CSV file
    # compute() performs the actual computation and returns a pandas DataFrame
    # to_csv writes the DataFrame to a CSV file
    unique_senders.compute().to_csv(output_file, index=False)

In [23]:
# Usage
extract_unique_senders('../data/merged_eth_transaction_data.csv', 'unique_senders.csv', 'Sender\'s Address')

In [2]:
# Directory where your files are stored
data_dir = "../data/"

# Read the CSV file into a DataFrame
df = dd.read_csv(f"{data_dir}unique_senders.csv")
# Format column names to replace spaces with '_' and replace single quotes with "".
df = df.rename(columns={col: col.lower().replace(' ', '_').replace("'", "") for col in df.columns})
# Overwrite the original CSV file
df.to_csv(f"{data_dir}unique_senders.csv", index=False, single_file=True)

['/Users/aeziz-local/Research/@2023/blockchain-JOM-special-issue-Sep30/ethereum-transaction-fee-mechanisms/data/unique_senders.csv']